# YT-Whisper-Scribe — Exécution sur Google Colab

Ce notebook prépare un environnement Colab (GPU, dépendances, ffmpeg), monte Google Drive pour persister les sorties, et exécute la CLI du projet.

Résumé des étapes:
- Activer le GPU dans Exécution > Modifier le type d'exécution > Accélérateur matériel: GPU.
- Installer dépendances (torch si besoin, whisper, yt-dlp, ffmpeg).
- Monter Google Drive et définir les chemins.
- Cloner le dépôt (ou placer le dossier du projet dans Drive).
- Lancer la transcription avec le GPU (CUDA).


In [ ]:
# Vérification GPU
import torch
print('CUDA dispo:', torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print('GPU:', torch.cuda.get_device_name(0))
    except Exception as e:
        print('Nom GPU non disponible:', e)
else:
    print('Activez le GPU: Exécution > Modifier le type d'exécution > Accélérateur: GPU')


In [ ]:
# Dépendances
# Colab inclut souvent torch avec CUDA. Si nécessaire, décommentez et adaptez la ligne torch ci-dessous.
# !pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!apt-get -y update && apt-get -y install ffmpeg
!pip install -U openai-whisper yt-dlp


In [ ]:
# Monte Google Drive pour persister sorties et caches
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Chemins et variables
from pathlib import Path
PROJECT_DIR = Path('/content/drive/MyDrive/YT-Whisper-Scribe')  # Modifiez si besoin
OUTPUT_DIR = PROJECT_DIR / 'data'  # Sorties persistées dans Drive
# Option: URL du dépôt si vous souhaitez cloner depuis GitHub
REPO_URL = ''  # ex: 'https://github.com/<user>/YT-Whisper-Scribe.git'
print('PROJECT_DIR =', PROJECT_DIR)
print('OUTPUT_DIR  =', OUTPUT_DIR)


In [ ]:
# Récupération du projet
import os, subprocess, sys
PROJECT_DIR.mkdir(parents=True, exist_ok=True)
if not any(PROJECT_DIR.iterdir()) and REPO_URL:
    # Clone le dépôt si le dossier est vide et qu'une URL est fournie
    print('Clonage du dépôt...', REPO_URL)
    subprocess.run(['git', 'clone', REPO_URL, str(PROJECT_DIR)], check=True)
elif not any(PROJECT_DIR.iterdir()) and not REPO_URL:
    print('Placez le dossier du projet dans', PROJECT_DIR, 'ou renseignez REPO_URL puis relancez cette cellule.')
# Crée le dossier de sortie
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
# (Option) Cache modèles sur Drive pour éviter les re-téléchargements
CACHE_BASE = PROJECT_DIR / '.cache'
os.makedirs(CACHE_BASE, exist_ok=True)
os.environ['XDG_CACHE_HOME'] = str(CACHE_BASE)
os.environ['TORCH_HOME'] = str(CACHE_BASE / 'torch')
print('Caches:', os.environ.get('XDG_CACHE_HOME'))


In [ ]:
%cd $PROJECT_DIR
!ls -la


## Exécuter la transcription

- Par défaut, le modèle est `turbo`, le device `cuda`, la langue `en`, le glossaire `SWOOD_Glossary.json`, et la sortie dans `data/`.
- Remplacez `URL_YOUTUBE` par l'URL souhaitée.


In [ ]:
# Exemple basique
!python scripts/transcribe.py "URL_YOUTUBE" --output_dir "$OUTPUT_DIR"


In [ ]:
# Exemple avancé avec vocabulaire personnalisé et sortie .txt
# Placez un vocabulaire (UTF-8, 1 terme/ligne) dans Drive, ex: $PROJECT_DIR/examples/vocab_example.txt
!python scripts/transcribe.py "URL_YOUTUBE" \
+  --output_dir "$OUTPUT_DIR" \
+  --output_format txt \
+  --vocab_file examples/vocab_example.txt
